If you have a ROMS grid file (e.g. `roms_grd.nc`), you can use ParaTC to generate a forcing file from a track dataframe.

Let's generate a storm as in `generating_2D_storms_with_TCModel.ipynb`:

In [ ]:
# Imports. We only need h80 for the class based approach
from paratc.tc_models import Holland1980 as h80
from paratc import make_example_data
import warnings 
warnings.filterwarnings('ignore')

# Open example track and generate example grid
track, grid_lon, grid_lat = make_example_data()

# Create storm instance and look at dataset
storm = h80( track, grid_lon, grid_lat, B_model='vickery00' )

# Scale winds to surface level
storm.scale_winds( 0.91 )

# Apply piecewise inflow angle model
storm.make_wind_vectors( inflow_model = 'wang20' )

# Add background winds using reciprocal relationship
storm.add_background_winds( bg_alpha = .55, bg_beta = 20 )

Now we can convert `storm.data` into the required format for ROMS forcing using `to_ROMS()`. This function requires a roms grid xarray dataset.

Using to_ROMS() will interpolate our storm vectors onto the ROMS U and V staggered grid. It will also rotate them to be aligned with the axes of the ROMS grid.

In [ ]:
# Open ROMS grid dataset
roms_grd = xr.open_dataset( 'roms_grd.nc' )

# Convert storm to format
storm.to_ROMS( roms_grd )
storm.to_netcdf( 'roms_frc.nc' )